In [ ]:
import numpy as np
import pandas as pd
import string
import xgboost as xgb
import io
import nltk
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('stopwords')
stopwords = stopwords.words('english')
stemmer = SnowballStemmer('english')

from textblob import TextBlob
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [18]:
tweets = pd.read_csv("train.csv", usecols=['id','text', 'target'])
test = pd.read_csv("test.csv", usecols=['id','text'])

In [ ]:
tweets.drop_duplicates(subset = 'text', keep = False, inplace = True)
tweets.info()

# Data loading.


In [ ]:
train_w_features = pd.read_csv('../TP2/train_features.csv')
test_w_features = pd.read_csv('../TP2/test_features.csv')

In [ ]:
features_only_train = train_w_features[['length','avg_word_length','amount_of_words','amount_of_unique_words','sentiment','stopwords_count','punctuation_count','mentions_count','hashtags_count','longest_word_length_without_stopwords','stopword_word_ratio','adjectives_count','nouns_count','verbs_count','adverbs_count']]
#features_only_train = tweets_metrics[['length','avg_word_length','amount_of_words','sentiment', 'stopwords_count', 'punctuation_count', 'longest_word_length_without_stopwords', 'amount_of_unique_words', 'hashtags_count', 'mentions_count']]

In [ ]:
features_only_test = test_w_features[['length','avg_word_length','amount_of_words','amount_of_unique_words','sentiment','stopwords_count','punctuation_count','mentions_count','hashtags_count','longest_word_length_without_stopwords','stopword_word_ratio','adjectives_count','nouns_count','verbs_count','adverbs_count']]
#features_only_test = test[['length','avg_word_length','amount_of_words','sentiment', 'stopwords_count', 'punctuation_count', 'longest_word_length_without_stopwords', 'amount_of_unique_words', 'hashtags_count', 'mentions_count']]

In [24]:
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [25]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding='utf8') as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix

def create_embedding_matrix_840(route, word_index, embedding_dim):
    embeddings_index = {}
    f = open(route, encoding='utf8')
    for line in f:
        values = line.split()
        word = ''.join(values[:-embedding_dim])
        coefs = np.asarray(values[-embedding_dim:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: # Si la palabra no esta queda llena de 0s
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

# Models.

### GridSearch.

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Concatenate, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Lambda, Activation, GaussianNoise, GaussianDropout
from keras import layers, Input, Model
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score
from tensorflow.keras.constraints import max_norm, unit_norm, min_max_norm
from tensorflow import keras

In [237]:
def create_conv1d(num_filters, kernel_size,vocab_size,embedding_dim,maxlen,batch_size,dense1_size,dense2_size):
    embedding = Embedding(vocab_size, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=True)
    x_train_input = Input(shape = (maxlen,), name = 'x_train_input')
    x_train_features_input = Input(shape = (15, ), name = 'x_features_train')
    emb = embedding(x_train_input)
    
    conv_out = Conv1D(num_filters, kernel_size, activation='relu')(emb)
    max_pool = GlobalMaxPooling1D()(conv_out)
    
    conc = Concatenate()([max_pool, x_train_features_input])
   
    dense1 = Dense(dense1_size, activation='relu')(conc)
    dense2 = Dense(dense2_size, activation='relu')(dense1)
    
    model = Model(inputs = [x_train_input , x_train_features_input], outputs = dense2)
    return model

def create_port_to_model(num_filters, kernel_size,vocab_size,embedding_dim,maxlen,batch_size,dense1_size,dense2_size):
    combi_input = Input(shape = (155,), name = 'port')
    input_train = Lambda(lambda x: x[:,:-15])(combi_input)
    input_features = Lambda(lambda x: x[:,140:])(combi_input)

    base_network = create_conv1d(num_filters, kernel_size,vocab_size,embedding_dim,maxlen,batch_size,dense1_size,dense2_size)
    processed = base_network([input_train,input_features])

    dense3 = Dense(1, activation='sigmoid')(processed)
    model = Model(combi_input,dense3)
    optimizer = Adam() #default
    model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model

In [238]:
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)

x_train = tweets_metrics['text'].values
y_train = tweets_metrics['target'].values
features = StandardScaler()
x_train_features = features.fit_transform(basic_features)

epochs = 15
embedding_dim = 100
maxlen = 140
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
vocab_size = len(tokenizer.word_index) + 1
x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)

embedding_matrix = create_embedding_matrix('Embeddings/glove.twitter.27B.100d.txt',tokenizer.word_index, embedding_dim)

In [239]:
param_grid = dict(num_filters=[128],
                      kernel_size=[7],
                      vocab_size=[vocab_size],
                      embedding_dim=[embedding_dim],
                      maxlen=[maxlen],
                      batch_size = [88],
                      dense1_size = [10],
                      dense2_size = [20,10])

model = KerasClassifier(build_fn=create_port_to_model,
                            epochs=epochs, validation_split=0.1,
                            verbose=1)

grid = GridSearchCV(estimator=model, param_grid=param_grid,
                              cv=4, verbose=2, n_jobs=1)

grid_result = grid.fit(np.concatenate((x_train,x_train_features), axis = 1), y_train, callbacks=[callback])

s = ('Best Accuracy : {:.4f}\n{}\n\n\n')
output_string = s.format(
            grid_result.best_score_,
            grid_result.best_params_)
print(output_string)

Fitting 4 folds for each of 2 candidates, totalling 8 fits
[CV] batch_size=88, dense1_size=10, dense2_size=20, embedding_dim=100, kernel_size=7, maxlen=140, num_filters=128, vocab_size=22811 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Train on 5017 samples, validate on 558 samples
Epoch 1/15
5017/5017 [==============================] - 9s 2ms/step - loss: 0.5171 - accuracy: 0.7473 - val_loss: 0.5379 - val_accuracy: 0.7509
Epoch 2/15
5017/5017 [==============================] - 10s 2ms/step - loss: 0.3783 - accuracy: 0.8385 - val_loss: 0.4833 - val_accuracy: 0.7652
Epoch 3/15
5017/5017 [==============================] - 9s 2ms/step - loss: 0.2915 - accuracy: 0.8882 - val_loss: 0.4813 - val_accuracy: 0.7796
Epoch 4/15
5017/5017 [==============================] - 8s 2ms/step - loss: 0.2236 - accuracy: 0.9197 - val_loss: 0.4517 - val_accuracy: 0.8047
Epoch 5/15
5017/5017 [==============================] - 9s 2ms/step - loss: 0.1452 - accuracy: 0.9569 -

In [242]:
pd.DataFrame(grid_result.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_dense1_size,param_dense2_size,param_embedding_dim,param_kernel_size,param_maxlen,param_num_filters,param_vocab_size,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,28.782677,10.218281,0.639944,0.032818,88,10,20,100,7,140,128,22811,"{'batch_size': 88, 'dense1_size': 10, 'dense2_...",0.789672,0.766541,0.771798,0.789020,0.779258,0.010261,2
1,36.360590,6.298348,0.661582,0.020972,88,10,10,100,7,140,128,22811,"{'batch_size': 88, 'dense1_size': 10, 'dense2_...",0.791286,0.786444,0.778256,0.785791,0.785444,0.004661,1


### Individual model.

In [26]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Concatenate, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Lambda, Activation, GaussianNoise, GaussianDropout
from keras import layers, Input, Model
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score
from tensorflow.keras.constraints import max_norm, unit_norm, min_max_norm
from tensorflow import keras

In [27]:
scaler = StandardScaler()
x_train_features = scaler.fit_transform(features_only_train)

x_train = train_w_features['text'].values
y_train = train_w_features['target'].values

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)

vocab_size = len(tokenizer.word_index) + 1
maxlen = 50
x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)

In [29]:
embedding_dim_1 = 100
embedding_matrix_1 = create_embedding_matrix('Embeddings/glove.twitter.27B.100d.txt',tokenizer.word_index, embedding_dim_1)

embedding_dim_2 = 300
embedding_matrix_2 = create_embedding_matrix_840('Embeddings/glove.840B.300d.txt',tokenizer.word_index, embedding_dim_2)

embedding_dim_3 = 200
embedding_matrix_3 = create_embedding_matrix('Embeddings/glove.6B.200d.txt',tokenizer.word_index, embedding_dim_3)

In [109]:
def create_conv1d(): #original
        
    embedding = Embedding(vocab_size, embedding_dim_1, input_length=maxlen, weights=[embedding_matrix_1], trainable=False)
    embedding2 = Embedding(vocab_size, embedding_dim_2, input_length=maxlen, weights=[embedding_matrix_2], trainable=True)
    embedding3 = Embedding(vocab_size, embedding_dim_3, input_length=maxlen, weights=[embedding_matrix_3], trainable=False)

    x_train_input = Input(shape = (maxlen,), name = 'x_train_input')
    x_train_features_input = Input(shape = (15, ), name = 'x_features_train')

    emb = embedding(x_train_input)
    emb2 = embedding2(x_train_input)
    emb3 = embedding3(x_train_input)

    #Emb 100
    #conv_out1_1 = Conv1D(128, 2, activation='relu')(emb)
    #activation1_1 = Activation('relu')(conv_out1_1)
    #max_pool1_1 = GlobalMaxPooling1D()(activation1_1)
    #conv_out1_2 = Conv1D(128, 3, activation='relu')(emb)
    #activation1_2 = Activation('relu')(conv_out1_2)
    #max_pool1_2 = GlobalMaxPooling1D()(activation1_2)

    #Emb 200
    conv_out2_1 = Conv1D(128, 2, activation='relu', kernel_constraint=max_norm(3), bias_constraint=max_norm(3))(emb2)
    activation2_1 = Activation('relu')(conv_out2_1)
    max_pool2_1 = GlobalMaxPooling1D()(activation2_1)
    conv_out2_2 = Conv1D(128, 3, activation='relu', kernel_constraint=max_norm(3), bias_constraint=max_norm(3))(activation2_1)
    activation2_2 = Activation('relu')(conv_out2_2)
    max_pool2_2 = GlobalMaxPooling1D()(activation2_2)

    #Emb 300
    #conv_out3_1 = Conv1D(128, 2, activation='relu')(emb3)
    #activation3_1 = Activation('relu')(conv_out3_1)
    #max_pool3_1 = GlobalMaxPooling1D()(activation3_1)
    #conv_out3_2 = Conv1D(128, 3, activation='relu')(emb3)
    #activation3_2 = Activation('relu')(conv_out3_2)
    #max_pool3_2 = GlobalMaxPooling1D()(activation3_2)

    conc = Concatenate()([max_pool2_2,  x_train_features_input])
    
    #noise1 = GaussianNoise(0.1)(conc)
    dense1 = Dense(100, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01))(conc)
    noise2 = GaussianNoise(0.1)(dense1)
    
    dense2 = Dense(10, activation='relu')(dense1)
    #noise1 = GaussianNoise(0.001)(dense2)
    dense3 = Dense(1, activation='sigmoid')(dense2)
    
    model = Model(inputs = [x_train_input , x_train_features_input], outputs = dense3)
    optimizer = Adam() #default
    model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    return model

In [112]:
model = create_conv1d()
history = model.fit([x_train,x_train_features], y_train,
                    epochs=3,
                    verbose=1,
                    batch_size=69)

Epoch 1/3
7434/7434 [==============================] - 21s 3ms/step - loss: 1.0982 - accuracy: 0.7818
Epoch 2/3
7434/7434 [==============================] - 23s 3ms/step - loss: 0.5130 - accuracy: 0.8608
Epoch 3/3
7434/7434 [==============================] - 22s 3ms/step - loss: 0.3117 - accuracy: 0.9089


In [113]:
result = pd.read_csv('perfect_submission.csv')
test_kagle = test_w_features[['id','text']]

x_test_kagle = test_w_features['text'].values
x_test_kagle = tokenizer.texts_to_sequences(x_test_kagle)
x_test_kagle = pad_sequences(x_test_kagle, padding='post', maxlen=maxlen)
x_test_features = features.transform(features_only_test)

submit_df = pd.DataFrame()
submit_df['id'] = test_kagle['id']
submit_df['prob'] = model.predict([x_test_kagle,x_test_features])
submit_df['target'] = submit_df['prob'].apply(lambda x: 0 if x < .5 else 1)

accuracy = accuracy_score(submit_df['target'], result['target'])
print(accuracy)

0.7925222188170395


In [ ]:
0.820410665032179

In [65]:
del submit_df['prob']
submit_df.to_csv("submit_prueba_45.csv", index=False)

### Evaluación de pre-trained words.

In [13]:
glove_embeddings = np.load('Embeddings/glove.840B.300d.pkl', allow_pickle=True)
fasttext_embeddings = np.load('Embeddings/crawl-300d-2M.pkl', allow_pickle=True)

In [15]:
import operator
def build_vocab(X):
    tweets = X.apply(lambda s: s.split()).values      
    vocab = {}
    for tweet in tweets:
        for word in tweet:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1                
    return vocab

def check_embeddings_coverage(X, embeddings):
    vocab = build_vocab(X)    
    covered = {}
    oov = {}    
    n_covered = 0
    n_oov = 0
    for word in vocab:
        try:
            covered[word] = embeddings[word]
            n_covered += vocab[word]
        except:
            oov[word] = vocab[word]
            n_oov += vocab[word]
            
    vocab_coverage = len(covered) / len(vocab)
    text_coverage = (n_covered / (n_covered + n_oov))
    sorted_oov = sorted(oov.items(), key=operator.itemgetter(1))[::-1]
    return sorted_oov, vocab_coverage, text_coverage

train_glove_oov, train_glove_vocab_coverage, train_glove_text_coverage = check_embeddings_coverage(tweets_metrics['text'], glove_embeddings)
test_glove_oov, test_glove_vocab_coverage, test_glove_text_coverage = check_embeddings_coverage(test['text'], glove_embeddings)
print('GloVe Embeddings cover {:.2%} of vocabulary and {:.2%} of text in Training Set'.format(train_glove_vocab_coverage, train_glove_text_coverage))
print('GloVe Embeddings cover {:.2%} of vocabulary and {:.2%} of text in Test Set'.format(test_glove_vocab_coverage, test_glove_text_coverage))

train_fasttext_oov, train_fasttext_vocab_coverage, train_fasttext_text_coverage = check_embeddings_coverage(tweets_metrics['text'], fasttext_embeddings)
test_fasttext_oov, test_fasttext_vocab_coverage, test_fasttext_text_coverage = check_embeddings_coverage(test['text'], fasttext_embeddings)
print('FastText Embeddings cover {:.2%} of vocabulary and {:.2%} of text in Training Set'.format(train_fasttext_vocab_coverage, train_fasttext_text_coverage))
print('FastText Embeddings cover {:.2%} of vocabulary and {:.2%} of text in Test Set'.format(test_fasttext_vocab_coverage, test_fasttext_text_coverage))

GloVe Embeddings cover 52.17% of vocabulary and 82.73% of text in Training Set
GloVe Embeddings cover 57.21% of vocabulary and 81.85% of text in Test Set
FastText Embeddings cover 51.63% of vocabulary and 81.88% of text in Training Set
FastText Embeddings cover 56.55% of vocabulary and 81.12% of text in Test Set


In [33]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

0.5652974442155101

### Verificación de misslabeled tweets individuales.

In [180]:
import csv
pa_checkear = test_w_features[['id','text']]
pa_checkear['target'] = result['target']
with open('perfect_submission.csv') as perfecto, open('submit_prueba_43.csv') as submit:
    perfect_csv = csv.reader(perfecto)
    submit_csv = csv.reader(submit)
    perf = {}
    subm = {}
    misseados = []
    for id,target in perfect_csv:
        perf[id] = perf.get(id,target)
    for id,target in submit_csv:
        subm[id] = subm.get(id,target)
    for id in subm:
        if subm[id] != perf[id]:
            misseados.append(id)
    for i in range(len(misseados)):
        misseados[i] = int(misseados[i])

In [184]:
len(misseados)

569

In [181]:
pa_checkear['wrong_label'] = pa_checkear['id'].apply(lambda x: x in misseados)
pa_checkear[pa_checkear['wrong_label'] == True].to_csv('mal_labeleados.csv', index = False)

In [193]:
pa_checkear[pa_checkear['wrong_label'] == True]['text'].str.contains('http').value_counts()

False    287
True     282
Name: text, dtype: int64